# Scraping dos arquivos de Microdados

este notebook realiza o download de todos os arquivos de microdados disponíveis (no site do IBGE)[https://www.ibge.gov.br/estatisticas/sociais/trabalho/22827-censo-demografico-2022.html?edicao=37225&t=microdados]. Os microdados estão separados por censo (2000 e 2010) e por UF.

In [12]:
import sys
import os
import requests
import zipfile
sys.path.append('../..')

# Data Manipulation
import pandas as pd

# from utils import *

from bs4 import BeautifulSoup as bs
import urllib.request

In [13]:
url = "https://www.ibge.gov.br/estatisticas/sociais/trabalho/22827-censo-demografico-2022.html?edicao=37225&t=microdados"

DATA_PATH = "../../data/"

# The following variables determine which cells will be run, default is true for all variables
config_vars = {
    'download_zips': True # Download zips from IBGE website
    ,'unzip_files': True # Unzip the .zip files
    move_up_files
    ,'rename_files': True # Rename files according to the definition {research}_{year}_{UF}.txt
}

## 1- Download dos Arquivos .zip

In [14]:
%%time
if config_vars['download_zips'] == True:
    census_list = []
    
    # scrape the html from IBGE's website
    soup = bs(requests.get(url).text)

    # find all unordered lists and iterate over them
    uls = soup.find_all("ul")
    for ul in uls:
    # find all links in the unordered list and iterate over them
        ul_as = ul.find_all("a")
        for ul_a in ul_as:
            # if one of the links refers to the state name "Acre", the list contains the census data
            if ul_a.text == "Acre":
                census_list.append(ul)

    # iterate over the census data list
    for census in census_list:
        # find all links in the census data list and iterate over them
        census_as = census.find_all("a")
        for census_a in census_as:
            # get the text of the link
            href = census_a['href']
            # get the year from the link's text
            idx_year = int(href.index("Censo_Demografico_") + len("Censo_Demografico_"))
            nm_year = href[idx_year:int(idx_year) + 4]
            # get the uf from the link's text
            nm_uf = href[href.rindex("/") + 1:-4]
            # define the filename
            nm_file = f"{DATA_PATH}/{nm_year}_{nm_uf}.zip"
            # download the file
            urllib.request.urlretrieve(href, nm_file)

CPU times: total: 7.33 s
Wall time: 2min 4s


## 2- Unzip dos arquivos

In [16]:
%%time
if config_vars['unzip_files'] == True:
    for file_path in os.listdir(DATA_PATH):
        if file_path.endswith('.zip'):
            with zipfile.ZipFile(DATA_PATH + file_path, 'r') as zip_ref:
                try:
                    zip_ref.extractall(DATA_PATH)
                    zip_ref.close()
                    os.remove(DATA_PATH + file_path)
                except Exception as e:
                    # print(f'File: {file_path} not processed due to Error: \n{e}')
                    pass
                

CPU times: total: 39.6 s
Wall time: 2min 20s


# 3- Renomeia os arquivos

In [ ]:
if config_vars['move_up_files'] == True:
    